In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [3]:
# Official Libraries
import json
import pandas as pd
import torch
from tqdm import tqdm

In [11]:
# Custom Libraries
from args import args
from prompts import entity_recognition
from utils import codexglue_helper
from utils.inference_helper import inference, update_file_name, reload_lib

In [12]:
class args(args):
    model_name = ["llama3_8b", "llama3_70b", "mistral_7b", "ibm_granite_3b", "ibm_granite_8b", "ibm_granite_20b", "ibm_granite_34b", "star_coder2_15b", "codellama_7b", "codellama_34b"][1]
    device = "cuda"
    device_auto = True
    

In [2]:
current_model = getattr(args, args.model_name)
print(current_model)
prompt = getattr(entity_recognition, args.model_name)

## Load the generated summaries
generated_summary = pd.read_csv('./saved_data/generated_summaries/generated_summaries_{}.csv'.format(args.model_name))
generated_summary = list(generated_summary['generated_summaries_{}'.format(args.model_name)])

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

if current_model['cached']:
    print("Using cached version")
    tokenizer = AutoTokenizer.from_pretrained(current_model["tokenizer"], cache_dir=args.cache_dir, device_map = 'auto')
    model = AutoModelForCausalLM.from_pretrained(current_model["model_path"], cache_dir=args.cache_dir, device_map = 'auto')

else:
    tokenizer = AutoTokenizer.from_pretrained(current_model["tokenizer"], device_map = 'auto')
    model = AutoModelForCausalLM.from_pretrained(current_model["model_path"], device_map = 'auto')


In [4]:
predicted_ner = []

for i in tqdm(range(len(generated_summary))):
    current_prompt = prompt.replace('{generated_summary}', generated_summary[i])
    
    with torch.no_grad():
        output = inference(tokenizer=tokenizer, model=model, prompt=current_prompt)

    predicted_ner.append(output)


### Saving the data

In [19]:

ner_out = pd.DataFrame()

ner_out['summaries_{}'.format(args.model_name)] = generated_summary
ner_out['predicted_ner_{}'.format(args.model_name)] = predicted_ner
ner_out.to_csv('./saved_data/entity_recognition_output/predicted_ner_{}.csv'.format(args.model_name), index=False)